In [ ]:
#
# multimodal image registration x,y,z,Tz batch
#

In [ ]:
import torch
from torchvision import transforms

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("You are using a GPU. You can go to the next cell.")
else:
  device = torch.device("cpu")
  print("You are using a CPU. You need to change runtime to GPU: Runtime > Change Runtime type.")

gpu


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import os
_colab_dir = "/content/gdrive/MyDrive/Colab Notebooks/segreg"
os.chdir(_colab_dir)
print(os.getcwd())

/content/gdrive/MyDrive/Colab Notebooks/segreg


In [ ]:
import csv
import glob
import os
import shutil
import subprocess
from dataclasses import dataclass

In [ ]:
@dataclass
class CSVData:
  mov_imgfile: str
  mov_segfile: str
  ref_imgfile: str
  ref_segfile: str
  ref_scale_var: float
  left_bound: int
  top_bound: int
  width: int
  height: int
  min_z: int
  max_z: int
  z_step: int
  tz_center: float

In [ ]:
def create_scan_xyzTz_command(directory, row_data):
    """
    Constructs a well-formatted and robust command for scan_xyzTz.py

    Args:
        directory: folder containing scan_xyzTz.py
        row_data
            mov_imgfile (str): Name of the reference image file.
            mov_seg_file (str): Name of the reference segmentation file.
            ref_imgfile (str): Name of the moving image file.
            ref_segfile (str): Name of the moving segmentation file.
            ref_scale_var (float): Scaling value limit of the reference image.
            xmi (int): Left bound pixel value of x search range.
            xma (int): Right bound pixel value of x search range.
            ymi (int): Top bound pixel value of y search range.
            yma (int): Bottom bound pixel value of y search range.
            zmi (int): Minimum z pxiel value of z search range.
            zma (int): Maximum z pxiel value of z search range.
            zstep (int): Step size for the z search.
            tz_center (float): Center value of the z angle search in degrees.
        Returns:
            str: The complete command string ready for execution.
    """

    command_args = [
        "python3",
        os.path.join(directory, "scan_xyzTz.py"),
        "-mov_imgfile", os.path.join(directory, row_data.mov_imgfile),
        "-mov_segfile", os.path.join(directory, row_data.mov_segfile),
        "-ref_imgfile", os.path.join(directory, row_data.ref_imgfile),
        "-ref_segfile", os.path.join(directory, row_data.ref_segfile),
        "-ref_scale_var", str(row_data.ref_scale_var),
        "-xmi", str(row_data.left_bound),
        "-xma", str(int(row_data.left_bound) + int(row_data.width)),
        "-ymi", str(row_data.top_bound),
        "-yma", str(int(row_data.top_bound) + int(row_data.height)),
        "-zmi", str(row_data.min_z),
        "-zma", str(row_data.max_z),
        "-zstep", str(row_data.z_step),
        "-angle", str(row_data.tz_center)
    ]

    return " ".join(command_args)

In [ ]:
mov_imgfile = '373_231009_SoRa_Blk_20x1x_img.tif'
mov_segfile = '373_231009_SoRa_Blk_20x1x_seg.tif'
ref_imgfile = '373_230614_A1_Blk_Reg2_40x_img.tif'
ref_segfile = '373_230614_A1_Blk_Reg2_40x_seg.tif'
ref_scale_var = 0
left_bound = 295
right_bound = 565
width = 90
height = 180
min_z = 10
max_z = 15
z_step = 1
tz_center = 0
results_dir = "results"
directory = './'

In [ ]:
row_data = CSVData(mov_imgfile, mov_segfile, ref_imgfile, ref_segfile, ref_scale_var, left_bound, right_bound, width, height, min_z, max_z, z_step, tz_center)

In [ ]:
results_dir += f"_{row_data.mov_segfile[:-4]}_{row_data.ref_imgfile[:-4]}"

In [ ]:
if not os.path.isdir(directory + results_dir):
  os.mkdir(directory + results_dir)

In [ ]:
cmd = create_scan_xyzTz_command(directory, row_data)

['python3', './scan_xyzTz.py', '-mov_imgfile', './373_231009_SoRa_Blk_20x1x_img.tif', '-mov_segfile', './373_231009_SoRa_Blk_20x1x_seg.tif', '-ref_imgfile', './373_230614_A1_Blk_Reg2_40x_img.tif', '-ref_segfile', './373_230614_A1_Blk_Reg2_40x_seg.tif', '-ref_scale_var', '0', '-xmi', '295', '-xma', '385', '-ymi', '565', '-yma', '745', '-zmi', '10', '-zma', '15', '-zstep', '1', '-angle', '0']


In [ ]:
subprocess.run(cmd, shell=True)

CompletedProcess(args='python3 ./scan_xyzTz.py -mov_imgfile ./373_231009_SoRa_Blk_20x1x_img.tif -mov_segfile ./373_231009_SoRa_Blk_20x1x_seg.tif -ref_imgfile ./373_230614_A1_Blk_Reg2_40x_img.tif -ref_segfile ./373_230614_A1_Blk_Reg2_40x_seg.tif -ref_scale_var 0 -xmi 295 -xma 385 -ymi 565 -yma 745 -zmi 10 -zma 15 -zstep 1 -angle 0', returncode=0)

In [ ]:
for tiffile in glob.glob("over*tif"):
  shutil.copy2(tiffile, os.path.join(directory + results_dir, tiffile))
  os.remove(tiffile)

for logfile in glob.glob("align_[xT]*_[1-9]*csv"):
  shutil.copy2(logfile, os.path.join(directory + results_dir, logfile))
  os.remove(logfile)